In [ ]:
# calculating miles traveled by unique scooters in June using functions.

In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import datetime as dt
%matplotlib inline
from math import sin, cos, sqrt, atan2, radians, asin
import numpy as np
from geopy.distance import geodesic


/Users/davidchung/.pyenv/versions/3.6.0/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/davidchung/.pyenv/versions/3.6.0/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
june = pd.read_csv('data/june.csv')

In [23]:
june_miles = pd.read_csv('data/june_scooter_miles.csv')

In [6]:
june.sumdgroup.unique()

array(['scooter', 'Scooter', 'bicycle'], dtype=object)

In [3]:
scooter = ['scooter','Scooter']
june =june[june['sumdgroup'].isin(scooter)]


In [4]:
scooter_id_list = june['sumdid'].tolist()

In [5]:
scoots_id = np.unique(scooter_id_list)

In [6]:
len(scoots_id)

7822

In [11]:
#must sort list by sumdid, in order to acquire acurate distance traveled.
june_concise = june[['latitude','longitude','sumdid','companyname']]
june_concise = june_concise.sort_values(by = 'sumdid')

In [12]:
def delta_lat(float_lat_list):
    i = 0
    total = 0

    while len(float_lat_list)-1 > i:
       total = total +abs(abs(float_lat_list[i+1])-abs(float_lat_list[i]))
       i= i + 1

    return total   

#function to calculate total changes in longitude
def delta_lng(float_lng_list):
    i = 0
    total = 0

    while len(float_lng_list)-1 > i:
       total = total + abs(abs(float_lng_list[i+1])-abs(float_lng_list[i]))
       i= i + 1

    return total   

def miles_traveled(lon1,lat1,lon2,lat2):
    
    lon3=radians(lon1)
    lat3=radians(lat1)
    lon4=radians(lon2) 
    lat4=radians(lat2)
    
    dlon = lon4 - lon3
    dlat = lat4 - lat3
    a = sin(dlat / 2) ** 2 + cos(lat3) * cos(lat4) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))*0.62

In [79]:
#initialize list and variable
sumdid_count = 0
scooter_traveled=[]
scooter_id = []
may_max_lat_list=[]

#while loop for assigning unique scooter id     b   
while len(scoots_id) > sumdid_count:
    
#it's only catching one data
   # str_scooter_id = str(scooter_id_list[sumdid_count])
    june2 = june_miles[(june_miles['sumdid'] == str(scoots_id[sumdid_count]))]#str_scooter_id)]

#intialize empty list
    lat_distance = [] #<- this is the line 
    lng_distance = [] #<-this is the line
    
 
    for item in june2['latitude']:
        lat_distance.append(float(item))     
    for item in june2['longitude']:
        lng_distance.append(float(item))
    
  

    change_lat = delta_lat(lat_distance)
    change_lng = delta_lng(lng_distance)

#function to calculate total miles traveled

    scooter_traveled.append(miles_traveled(0,0,change_lng,change_lat))
    scooter_id.append(scoots_id[sumdid_count])
    #print(scooter_traveled)
    sumdid_count = sumdid_count + 1


In [7]:
#calculate average positions of each scooter

def average_position(position):
    
    sum = 0
    
    for num in position:
        sum = sum + num
    
    return sum/len(position)




In [11]:
#average lat and lng for low mile scooters, since they are not 
#moving alot averaging position to make it one position in map
sumdid_count = 0
scooter_average_position_lat = []
scooter_average_position_lng = []
scooter_id=[]
while len(scoots_id)>sumdid_count:
    
   # create list of specific scooter id
    june_list = june[(june['sumdid'] == str(scoots_id[sumdid_count]))]#str_scooter_id)]

    
    #initialize empty list
    lat_distance = [] #<- this is the line 
    lng_distance = [] #<-this is the line
    
     #iterate through each scooter's lat and lng and add to array
    for item in june_list['latitude']:
        lat_distance.append(float(item))     
    for item in june_list['longitude']:
        lng_distance.append(float(item))
    #calculate average lat and lng for each scooter
    average_lat_pos = average_position(lat_distance)
    average_lng_pos = average_position(lng_distance)          
    
    #add average lat and lng into new list
    scooter_average_position_lat.append(float(average_lat_pos))
    scooter_average_position_lng.append(float(average_lng_pos))
    scooter_id.append(scoots_id[sumdid_count])
    sumdid_count = sumdid_count+1
    

In [12]:
scooter_id = pd.DataFrame(scooter_id)
scooter_lat = pd.DataFrame(scooter_average_position_lat)
scooter_lng = pd.DataFrame(scooter_average_position_lng)

scooter_id = scooter_id.rename(columns = {0:'sumdid'})
scooter_lat = scooter_lat.rename(columns={0:'latitude'})
scooter_lng = scooter_lng.rename(columns={0:'longitude'})

temp = pd.merge(scooter_id,scooter_lat,how='outer',left_index=True,right_index=True)
inactive_scooter_pos = pd.merge(temp,scooter_lng,how='outer',left_index=True,right_index=True)
inactive_scooters = pd.merge(inactive_scooter_pos,low_miles_scooters,on=['sumdid'],how='outer')

In [13]:
low_miles_scooters = pd.read_csv('data/june_scooter_miles.csv')

In [14]:
inactive_scooters = pd.merge(inactive_scooter_pos,low_miles_scooters,on=['sumdid'],how='outer')

In [17]:
inactive_scooters.to_csv('data/inactive_scooters_position_june.csv')

In [19]:
inactive_scooters.head()

,sumdid,latitude,longitude,Unnamed: 0,Miles_traveled
0,Powered-017d3133-f14a-2b83-ee4f-d777e7c5b619,36.154141,-86.781196,0,56.542984
1,Powered-01a24436-0315-e1bb-7ce0-d081d05dff7d,36.161103,-86.787872,1,51.545579
2,Powered-03be23ca-d43b-222f-be54-e44b5b4690df,36.156918,-86.777882,2,66.012982
3,Powered-046201fb-6532-1f37-6334-3612fb1e61f7,36.158447,-86.782852,3,81.440421
4,Powered-0479bb84-afbd-0426-f1c4-df628542a88c,36.167397,-86.769893,4,62.884069
